In [1]:
# Import required libraries
import arcpy
import tqdm
#import re
from arcpy import *
from arcpy.sa import *
from arcpy.ia import *
from arcpy.conversion import *
#import shutil
import os
import math
import gc
#import glob
from tqdm import tqdm
from datetime import datetime

# Setting up working environment
env.overwriteOutput = True
env.parallelProcessingFactor = "90%"

# Setting the study area

# A list of string containing the name of the study area
locations = ['MOKAU']

# A list of the height of Mean High Water Spring in cm according to the sequence of study area defined above
tide = [130]

##### DIRECTIONS FOR USAGE

## 1. PREPARE SOURCE FILE

#  i.    Multi-spectral images to be placed in {source_dir}\RGBI_Tifs\
#  ii.   A folder with the name of the study area in all capital letters without space
#  iii.  A Digital Elevation Model named as "DEM.tif" placed in {source_dir}\{name of study area}\SOURCE\
#  iv.   A Digital Surface Model named as "DSM.tif" placed in {source_dir}\{name of study area}\SOURCE\
#  v.    A polygon featuer layer confining the study area with a field names "NAME" for the file name of the multispectral images
#        named as "study_area.shp" to be placed in {source_dir}\{name of study area}\SOURCE\
#  vi.   A polyline featuer layer indicated the flow of water under the hanging structure (i.e. a bridge),
#        named as "flow_line.shp" to be placed in {source_dir}\{name of study area}\SOURCE\
#  vii.  A polyline featuer layer indicated the origin of water coming in,
#        named as "water_origin.shp" to be placed in {source_dir}\{name of study area}\SOURCE\
#  viii. A featuer layer containing the groud truth data for land cover classification,
#        named as "training_polygon.shp" to be placed in {source_dir}\{name of study area}\SOURCE\
#  ix.   A featuer layer containing the groud truth data for land cover classification,
#        named as "validation_polygon.shp" to be placed in {source_dir}\{name of study area}\SOURCE\

ModuleNotFoundError: No module named 'arcpy'

In [ ]:
# Print current system time
def printTime(fn):
    print(fn +' at ' +  datetime.now().strftime("%H:%M:%S"))

In [ ]:
# Copy source image to destination folder

def copyImage(file_list):

    img_list = []
    
    printTime('Copying files started')
    for i in tqdm(range(len(file_list))):
        source_img = 'D:\\AERIAL_IMAGES\\RGBI_Tifs\\RGBI_'+file_list[i]+'.tif'
        img_list.append(source_img)
        del(source_img)
        gc.collect()
    
    
    MosaicToNewRaster_management(img_list, output_dir, "RGBI_WHOLE.tif",'' ,'' ,'' , 4)
    
    rgbi_whole = Raster(output_dir + "RGBI_WHOLE.tif")
    rgbi_whole = ExtractByMask(rgbi_whole, tile_index, "INSIDE")
    rgbi_whole.save( output_dir + "RGBI_WHOLE.tif")
    
    # copy the source DEM & DSM to output folder
    s_dem = Raster(source_dir + 'DEM.tif')
    s_dem.save( output_dir + 'DEM.tif')
    s_dsm = Raster(source_dir + 'DSM.tif')
    s_dsm.save( output_dir + 'DSM.tif')
    del(img_list, s_dem, s_dsm)
    gc.collect()
    
    printTime('copyImage done')
    

# Calculate Compound Topographic Index ( CTI / TWI )

#####    CTI = ln ( $\alpha$ / tan $\beta$ )
#####    where
#####    $\alpha$ = ( Flow accumulation area + 1 ) * cellsize
#####    $\beta$ = $Slope^o$ * ( $\pi$ / 2) / 90

In [ ]:
def getTWI():

    SLOPE = Raster( output_dir + 'SLOPE.tif' )
    flow_accu = Raster(output_dir + "FLW_ACCU.tif")
    alpha = ( flow_accu + 1 ) * 1
    beta = SLOPE * ( math.pi / 2 ) / 90

    TWI = Ln( alpha / Tan(beta) ) 
    TWI.save( output_dir + 'TWI.tif' )

    del(alpha, beta, TWI, SLOPE)
    gc.collect()
    printTime('getTWI done')

In [ ]:
# Prepare NDVI

def getNDVI():
        
    rgbi_whole = Raster(output_dir + "RGBI_WHOLE.tif")
    ndvi = NDVI(Raster(rgbi_whole), 4, 1)
    ndvi.save( output_dir + "NDVI.tif" )
    del(ndvi, rgbi_whole)
    gc.collect()
    printTime('getNDVI done')
    

In [ ]:
# Prepare NDWI

def getNDWI():
        
    rgbi_whole = Raster(output_dir + "RGBI_WHOLE.tif")
    bandR = ExtractBand(rgbi_whole, None, "Band_1")
    bandG = ExtractBand(rgbi_whole, None, "Band_2")
    bandB = ExtractBand(rgbi_whole, None, "Band_3")
    bandNIR = ExtractBand(rgbi_whole, None, "Band_4")
    
    NDWI = (bandG - bandNIR) / (bandG + bandNIR)
    NDWI.save( output_dir + "NDWI.tif")
    
    del(rgbi_whole, bandR, bandG, bandB, bandNIR, NDWI)
    gc.collect()
    
    printTime('getNDWI done')

In [ ]:
# prepare EVI - Enhanced Vegetation Index

def getEVI():
    
    rgbi_whole = Raster(output_dir + "RGBI_WHOLE.tif")
    bandR = ExtractBand(rgbi_whole, None, "Band_1")
    bandG = ExtractBand(rgbi_whole, None, "Band_2")
    bandB = ExtractBand(rgbi_whole, None, "Band_3")
    bandNIR = ExtractBand(rgbi_whole, None, "Band_4")
    
    EVI = (2.5 * (bandNIR - bandR)) / (bandNIR + (2.4 * bandR) + 10000)
    EVI.save( output_dir + "EVI.tif")
    
    del(rgbi_whole, bandR, bandG, bandB, bandNIR, EVI)
    gc.collect()
    
    printTime('getEVI done')

In [ ]:
# prepare GCI - Green Chlorophyll Index

def getGCI():
    
    rgbi_whole = Raster(output_dir + "RGBI_WHOLE.tif")
    bandR = ExtractBand(rgbi_whole, None, "Band_1")
    bandG = ExtractBand(rgbi_whole, None, "Band_2")
    bandB = ExtractBand(rgbi_whole, None, "Band_3")
    bandNIR = ExtractBand(rgbi_whole, None, "Band_4")
    
    GCI = (bandNIR / bandG) - 1
    GCI.save( output_dir + "GCI.tif")
    
    del(rgbi_whole, bandR, bandG, bandB, bandNIR, GCI)
    gc.collect()
    
    printTime('getGCI done')

In [ ]:
# prepare GLI - Green Leaf Index

def getGLI():
    
    rgbi_whole = Raster(output_dir + "RGBI_WHOLE.tif")
    bandR = ExtractBand(rgbi_whole, None, "Band_1")
    bandG = ExtractBand(rgbi_whole, None, "Band_2")
    bandB = ExtractBand(rgbi_whole, None, "Band_3")
    bandNIR = ExtractBand(rgbi_whole, None, "Band_4")
    
    GLI = ((bandG - bandR) + (bandG - bandB)) / ((2 * bandG) + bandR + bandB)
    GLI.save( output_dir + "GLI.tif")
    
    del(rgbi_whole, bandR, bandG, bandB, bandNIR, GLI)
    gc.collect()
    
    printTime('getGLI done')

In [ ]:
# prepare SAVI - Soil Adjusted Vegetation Index

def getSAVI():
    
    rgbi_whole = Raster(output_dir + "RGBI_WHOLE.tif")
    bandR = ExtractBand(rgbi_whole, None, "Band_1")
    bandG = ExtractBand(rgbi_whole, None, "Band_2")
    bandB = ExtractBand(rgbi_whole, None, "Band_3")
    bandNIR = ExtractBand(rgbi_whole, None, "Band_4")
    L = 0.5 #Cobertura vegetacion 0-1
    
    SAVI = ((bandNIR - bandR) / (bandNIR + bandR + L))* (1+L)
    SAVI.save( output_dir + "SAVI.tif")
    
    del(rgbi_whole, bandR, bandG, bandB, bandNIR, SAVI)
    gc.collect()
    
    printTime('getSAVI done')

In [ ]:
# Calculate all necessary topographic attributes
# i.e. Vegetation height, Slope, Flow direction, Flow accumulation and Basin

def getTopoAttr():
    DEM = Raster(output_dir+"DEM.tif")
    DSM = Raster(output_dir+"DSM.tif")
    
    # Create a layer of vegetaion height
    VEGE_HEIGHT = Raster(DSM) - Raster(DEM)
    VEGE_HEIGHT.save( output_dir + 'VEGE_HEIGHT.tif')
    
    VH_SHORTGRASS = Con(VEGE_HEIGHT, VEGE_HEIGHT, None, "Value >=0 AND Value <= 0.05 ") 
    VH_SHORTGRASS.save( output_dir + 'VH_SHORTGRASS.tif')
    VH_LONGGRASS = Con(VEGE_HEIGHT, VEGE_HEIGHT, None, "Value >0.05 AND Value <= 0.5 ") 
    VH_LONGGRASS.save( output_dir + 'VH_LONGGRASS.tif')
    VH_SHRUB = Con(VEGE_HEIGHT, VEGE_HEIGHT, None, "Value >0.5 AND Value <= 2 ") 
    VH_SHRUB.save( output_dir + 'VH_SHRUB.tif')
    VH_TREES = Con(VEGE_HEIGHT, VEGE_HEIGHT, None, "Value >2 ") 
    VH_TREES.save( output_dir + 'VH_TREES.tif')

    # Create a slope for later analysis
    SLOPE = Slope(DEM, 'DEGREE')
    SLOPE.save( output_dir + 'SLOPE.tif')

    # Fill the DEM and create a flow direction
    filled_dem = Fill(DEM)
    flow_dir = FlowDirection(filled_dem, 'NORMAL', '' , 'D8')
    flow_dir.save( output_dir + 'FLW_DIR.tif' )

    # Create the flow accumulation from flow direction
    flow_accu = FlowAccumulation(flow_dir,'','','D8')
    flow_accu.save( output_dir + 'FLW_ACCU.tif' )

    # Create a basin from the flow direction
    BASIN = Basin(flow_dir)
    BASIN.save( output_dir + 'BASIN.tif' )
    
    del(VEGE_HEIGHT, VH_SHORTGRASS, VH_LONGGRASS, VH_SHRUB, VH_TREES, SLOPE, flow_dir, flow_accu, BASIN, DEM, DSM)
    
    printTime('getTopoAttr done')

In [ ]:
# Segmentation before classification

def segmentation():
    RGB = Raster( output_dir + 'RGBI_WHOLE.tif')
    SEG_RGBI = SegmentMeanShift(RGB, '19.0', '15.0', '15')
    SEG_RGBI.save(output_dir + "SEG_RGBI.tif")
    
    del(RGB, SEG_RGBI)
    gc.collect()
    
    printTime('segmentation done')

In [ ]:
# Merge all bands into a single raster

def compositeBands():
    
    DEM = Raster(output_dir+"DEM.tif")
    DSM = Raster(output_dir+"DSM.tif")
    VEGE_HEIGHT = Raster(output_dir+"VEGE_HEIGHT.tif")
    SLOPE = Raster(output_dir+"SLOPE.tif")
    TWI = Raster(output_dir+"TWI.tif")
    NDVI = Raster(output_dir+"NDVI.tif")
    NDWI = Raster(output_dir+"NDWI.tif")
    EVI = Raster(output_dir+"EVI.tif")
    GCI = Raster(output_dir+"GCI.tif")
    GLI = Raster(output_dir+"GLI.tif")
    SAVI = Raster(output_dir+"SAVI.tif")
    DIS_ACU = Raster(output_dir  + "\\Inundation\\DA\\DA_" + str(tidelevel_inCM) + "CMtide.tif")
    TIDAL_DEP = Raster(output_dir + "\\Inundation\\depth_" + str(tidelevel_inCM) + "CMtide.tif")
    
    #RGBI_WHOLE = Raster(output_dir+"SEG_RGBI.tif")
    RGBI_WHOLE = Raster( output_dir + 'RGBI_WHOLE.tif')
    management.CompositeBands([DEM,DSM,VEGE_HEIGHT,SLOPE,TWI,NDVI,NDWI,EVI,GCI,GLI,SAVI,DIS_ACU,TIDAL_DEP,RGBI_WHOLE], output_dir + "COMBINED.tif")
    #management.CompositeBands([DEM,DSM,VEGE_HEIGHT,SLOPE,TWI,RGBI_WHOLE], output_dir + "COMBINED.tif")
    
    COMBINED_RASTER = Raster(output_dir + "COMBINED.tif")
    COMBINED_RASTER.renameBand(1, 'DEM')
    COMBINED_RASTER.renameBand(2, 'DSM')
    COMBINED_RASTER.renameBand(3, 'VEGE_HEIGHT')
    COMBINED_RASTER.renameBand(4, 'SLOPE')
    COMBINED_RASTER.renameBand(5, 'TWI')
    COMBINED_RASTER.renameBand(6, 'NDVI')
    COMBINED_RASTER.renameBand(7, 'NDWI')
    COMBINED_RASTER.renameBand(8, 'EVI')
    COMBINED_RASTER.renameBand(9, 'GCI')
    COMBINED_RASTER.renameBand(10, 'GLI')
    COMBINED_RASTER.renameBand(11, 'SAVI')
    COMBINED_RASTER.renameBand(12, 'DIS_FMTIDE')
    COMBINED_RASTER.renameBand(13, 'TIDAL_DEPT')
    COMBINED_RASTER.renameBand(14, 'RED')
    COMBINED_RASTER.renameBand(15, 'GREEN')
    COMBINED_RASTER.renameBand(16, 'BLUE')
    COMBINED_RASTER.renameBand(17, 'NIR')
    
    del(VEGE_HEIGHT,SLOPE,TWI,NDVI,NDWI,EVI,GCI,GLI,SAVI,DIS_ACU,TIDAL_DEP,RGBI_WHOLE,COMBINED_RASTER, DEM, DSM)
    #del(VEGE_HEIGHT,SLOPE,TWI,RGBI_WHOLE,COMBINED_RASTER, DEM, DSM)
    gc.collect()
    
    printTime('compositeBands done')
    

In [ ]:
# Training of Random Trees Classifier to create a .ecd definition file


def trainRT_classifier():
    training_polygon = source_dir + 'training_polygon.shp'
    
    input_raster = Raster( output_dir + "COMBINED.tif")
    classifier_definition = output_dir + 'rt_classifier.ecd'
    
    TrainRandomTreesClassifier(input_raster, training_polygon, classifier_definition, None, 300, 100, 1000)
    
    del(input_raster)
    gc.collect()
    
    printTime('trainRT_classifier done')

In [ ]:
# Perform a supervised classification by Random Trees

def classify_combined():
    
    input_raster = Raster( output_dir + "COMBINED.tif")
    classifier_definition = output_dir + 'rt_classifier.ecd'
        
    classified = Classify(input_raster, None, classifier_definition)
    classified = ExtractByMask(classified, tile_index, "INSIDE")
    classified.save( output_dir + "RT_CLASSIFIED.tif")
    
    del(input_raster, classified)
    gc.collect()
    
    printTime('classify_combined done')

In [ ]:
# Perform accuracy assessment by calculate confusion matrix

def accuracy_assessment():
    
    ## Create accuracy assessment point
    validation_polygon = source_dir + 'validation_polygon.shp'
    validation_points = output_dir + 'validation_points.shp'
    CreateAccuracyAssessmentPoints(validation_polygon, validation_points, "GROUND_TRUTH", 1000, "EQUALIZED_STRATIFIED_RANDOM")
    
    ## Update the assessment points
    classified_img = output_dir + "RT_CLASSIFIED.tif"
    assessment_points = output_dir + "assessment_points.shp"
    
    UpdateAccuracyAssessmentPoints(classified_img, validation_points, assessment_points)    
    
    ## Compute Confusion matrix
    matrix = output_dir + 'confusion_matrix.dbf'
    #arcpy.DeleteRows_management(matrix)
    ComputeConfusionMatrix(assessment_points, matrix)
    
    del(validation_polygon, validation_points, matrix, assessment_points)
    gc.collect()
    
    printTime('accuracy_assessment done')

In [ ]:
# Burn Flow line into DEM

def burnFlowLine():
    DEM = Raster(output_dir+"DEM.tif")
    DSM = Raster(output_dir+"DSM.tif")
    
    # checking if the directory exists, 
    if not os.path.exists(output_dir + "\\Inundation\\hydroAdaptation\\"):
        os.makedirs(output_dir + "\\Inundation\\hydroAdaptation\\")           # if not, create one
    
    temp_dem = DEM
    temp_dem.save(output_dir + "\\Inundation\\hydroAdaptation\\temp_dem.tif")
    arcpy.env.extent = temp_dem
    flow_line = source_dir + "flow_line.shp"
    flow_endpoints = output_dir + "\\Inundation\\hydroAdaptation\\flow_endpoints.shp"
    
    endPoint_DEM = output_dir + "Inundation\\hydroAdaptation\\endPoint_DEM.shp"
    
    endPoint_stats = output_dir + "Inundation\\hydroAdaptation\\endPoint_stats.dbf"
    temp_Raster = output_dir + "\\Inundation\\hydroAdaptation\\temp_Raster.tif"
    
    management.FeatureVerticesToPoints(flow_line, flow_endpoints, "BOTH_ENDS")
    ExtractValuesToPoints(flow_endpoints, temp_dem, endPoint_DEM, "NONE", "VALUE_ONLY")   
    
    arcpy.analysis.Statistics(endPoint_DEM, endPoint_stats, [["RASTERVALU", "MAX"]], "ORIG_FID")
    
    hydro_joined = management.AddJoin(flow_line, "FID", endPoint_stats, "ORIG_FID","KEEP_ALL")
    
    
    arcpy.conversion.PolylineToRaster(hydro_joined, "endPoint_stats.MAX_RASTER", temp_Raster, "MAXIMUM_LENGTH", None, DEM, "BUILD")
        
    temp_Raster2 = Con( IsNull(temp_Raster),9999,temp_Raster)
    temp_Raster2.save (output_dir +"\\Inundation\\hydroAdaptation\\temp_Raster2.tif")
    
    adapted_DEM = Con(LessThan(temp_Raster2, 9999), temp_Raster2, temp_dem)
    adapted_DEM.save(output_dir + "adapted_DEM.tif")
    
    del(temp_dem,flow_line,flow_endpoints,endPoint_DEM,endPoint_stats,temp_Raster,hydro_joined,temp_Raster2,adapted_DEM, DEM, DSM)
    gc.collect()
    
    printTime('burnFlowLine done')

In [ ]:
# Create inundation model
# Inundation model.
# Credit to Thomas Balstrøm, Nov. 9 2021.
# Dept. of Geosciences, Univ. of Copenhagen
# tb@ign.ku.dk

def createInundation():
    
    DEM = Raster(output_dir+"DEM.tif")
    DSM = Raster(output_dir+"DSM.tif")
    
    # checking if the directory exists, 
    if not os.path.exists(output_dir + "\\Inundation\\BD\\"):
        os.makedirs(output_dir + "\\Inundation\\BD\\")           # if not, create one
    
    # checking if the directory exists, 
    if not os.path.exists(output_dir + "\\Inundation\\DA\\"):
        os.makedirs(output_dir + "\\Inundation\\DA\\")           # if not, create one
        
    adapted_DEM = Raster(output_dir + "adapted_DEM.tif")
    water_origin = source_dir + "water_origin.shp"
    
    in_cost_raster = SetNull(adapted_DEM,1,'Value >=' + str(tidelevel_inCM/100))
    BD_CM = output_dir  + "\\Inundation\\BD\\BD_" + str(tidelevel_inCM) + "CMtide.tif"

    this_DA = DistanceAccumulation(water_origin, "", "", in_cost_raster, "", "BINARY 1 -30 30", "", "BINARY 1 45", BD_CM, "", "", "", "", "", "", "PLANAR")
    this_DA.save(output_dir  + "\\Inundation\\DA\\DA_" + str(tidelevel_inCM) + "CMtide.tif")

    this_inundated = Con(this_DA,tidelevel_inCM)
    this_inundated.save(output_dir  + "\\Inundation\\inundated_"+ str(tidelevel_inCM) + "CMtide.tif")
    management.BuildPyramids(this_inundated)

    inundation_depth = (this_inundated/100.0) - DEM
    inundation_depth.save(output_dir + "\\Inundation\\depth_" + str(tidelevel_inCM) + "CMtide.tif")
  
    del(in_cost_raster,BD_CM,this_DA,this_inundated,inundation_depth,DEM,DSM)
    gc.collect()
    
    printTime('createInundation done')


In [ ]:
# Predict the spawning location for study area

def predictLocations():
    
    DEM = Raster(output_dir+"DEM.tif")
    DSM = Raster(output_dir+"DSM.tif")
    VEGE_HEIGHT = Raster(output_dir+"VEGE_HEIGHT.tif")
    SLOPE = Raster(output_dir+"SLOPE.tif")
    TWI = Raster(output_dir+"TWI.tif")
    
    ndvi= Raster( output_dir + "NDVI.tif" )
    NDWI= Raster( output_dir + "NDWI.tif" )
    EVI= Raster( output_dir + "EVI.tif" )
    GCI= Raster( output_dir + "GCI.tif" )
    GLI= Raster( output_dir + "GLI.tif" )
    SAVI= Raster( output_dir + "SAVI.tif" )
    
    INUNDATED_DEPTH = Raster( output_dir + "\\Inundation\\depth_" + str(tidelevel_inCM) + "CMtide.tif" )
    INUNDATEDAREA = Raster( output_dir + "\\Inundation\\inundated_"+ str(tidelevel_inCM) + "CMtide.tif" )
    DIS_ACU = Raster( output_dir+"\\Inundation\\DA\\DA_"+ str(tidelevel_inCM) + "CMtide.tif" )
    VEGETATION = Raster( output_dir + "RT_CLASSIFIED.tif" )
    SUIT_VEG = Con(VEGETATION, VEGETATION, None, "Class_name LIKE '%Grass%' OR Class_name LIKE '%Shrub%'")
    POTENTIAL_SITE_INPOINT = output_dir + "POTENTIAL_SITE_INPOINT.shp"
    

 
    # Use Con() to create the output raster based on the combined conditions
    
        # - Fairly gentle slope ( less than 5o )
        # - With suitable vegetation ( Grassland and shrub )
        # - Suitable vegetation height ( 5cm to 5m )
        # - Shallow depth of inundation at MHWS ( 3cm to 20cm )

    prediction = Con(( (SLOPE<=5) & ( VEGE_HEIGHT >=0.05 ) & ( VEGE_HEIGHT <= 5) & (INUNDATED_DEPTH>=0.03) & (INUNDATED_DEPTH<=0.2) & (SUIT_VEG>=0) ), INUNDATEDAREA)
    prediction.save( output_dir + "PREDICTED_LOCATION.tif" )
    management.BuildPyramids(prediction)
    
    # Convert the potential spawning location to point feature for regression prediction
    arcpy.conversion.RasterToPoint(prediction, POTENTIAL_SITE_INPOINT)
    ExtractMultiValuesToPoints(POTENTIAL_SITE_INPOINT, [DEM, DSM, VEGE_HEIGHT, SLOPE, TWI, ndvi, NDWI, EVI, GCI, GLI, SAVI, [INUNDATED_DEPTH, "TIDAL_DEPT"], [DIS_ACU, "DIS_FMTIDE"]], "NONE")

    del(DEM, DSM, VEGE_HEIGHT, SLOPE, TWI, INUNDATED_DEPTH, INUNDATEDAREA, VEGETATION, POTENTIAL_SITE_INPOINT)
    gc.collect()
    
    printTime('predictLocations done')

In [ ]:
def FBCR_predict():
    
    prediction_type = 'PREDICT_FEATURES'
    in_features = "D:\\PROJECT_WHITEBAIT\\PORT_WAIKATO\\OUTPUT\\SPAWNING_RECORD.shp"
    variable_predict = 'Spawning'
    treat_variable_as_categorical = 'CATEGORICAL'
    explanatory_variables = [["EVI", "false"], ["GCI", "false"], ["GLI", "false"], ["NDVI", "false"], ["NDWI", "false"], 
                             ["SAVI", "false"], ["SLOPE", "false"], ["TWI", "false"], ["VEGE_HEIGH", "false"], 
                             ["TIDAL_DEPT", "false"], ["DIS_FMTIDE","false"]]
    distance_features = None
    explanatory_rasters = None
    features_to_predict = output_dir + "POTENTIAL_SITE_INPOINT.shp"
    output_features = output_dir + "FBCR_PREDICTED_SITE.shp"
    output_raster = None
    explanatory_variable_matching = None
    explanatory_distance_matching = None
    explanatory_rasters_matching = None
    output_trained_features = output_dir + "FBCR_TRAINEDFEATURE.shp"
    output_importance_table = output_dir + "FBCR_VARIABLEIMPORTANCE.dbf"
    use_raster_values = False
    number_of_trees = 100
    minimum_leaf_size = None
    maximum_depth = None
    sample_size = None
    random_variables = None
    percentage_for_training = None
    output_classification_table = output_dir + "FBCR_CLASSTABLE.dbf"
    output_validation_table = output_dir + "FBCR_VALIDATIONTABLE.dbf"
    compensate_sparse_categories = None
    number_validation_runs = None
    calculate_uncertainty = None
    output_trained_model = "D:\\PROJECT_WHITEBAIT\\PORT_WAIKATO\\OUTPUT\\FBCR_TRAINEDMODEL.ssm"

    stats.Forest(prediction_type, in_features, variable_predict, treat_variable_as_categorical, explanatory_variables, distance_features, explanatory_rasters, features_to_predict, output_features, output_raster, explanatory_variable_matching, explanatory_distance_matching, explanatory_rasters_matching, output_trained_features, output_importance_table, use_raster_values, number_of_trees, minimum_leaf_size, maximum_depth, sample_size, random_variables, percentage_for_training, output_classification_table, output_validation_table, compensate_sparse_categories, number_validation_runs, calculate_uncertainty, output_trained_model)
    
    printTime("FBCR_predict Done")

In [ ]:
def main():
    
    # Reading all the required file name from an index file
    rows = SearchCursor(tile_index)   
    file_list = []   # Put all required file name into a list
    for row in rows:
        file_list.append(row.getValue('NAME'))
    
    ####################### Performing GIS analysis #################################
    
    copyImage(file_list)

    printTime('Computing topographic attributes started')
    getTopoAttr()
    printTime('Computing TWI started')
    getTWI()
    printTime('Computing NDVI started')
    getNDVI()
    printTime('Computing NDWI started')
    getNDWI()
    printTime('Computing EVI started')
    getEVI()
    printTime('Computing GCI started')
    getGCI()
    printTime('Computing GLI started')
    getGLI()
    printTime('Computing SAVI started')
    getSAVI()    
    
    ######################## Create inundation model #################################

    # Create a flow line adapted DEM
    printTime("Create flow line adapted DEM started")
    burnFlowLine()

    printTime("Create inundation model started")
    createInundation()

    # Mosaic all attribute into a single raster

    #printTime('Segmentation of the aerial image')
    #segmentation()

    printTime('Composite all bands into one raster started')
    compositeBands()

    # Perform Classification
    printTime('Training of Random Trees Classifier started')
    trainRT_classifier()

    printTime("Classify images started")
    classify_combined()

    printTime("Calculate confusion matrix started")
    accuracy_assessment()
    
    ################################# Prediction on potential spawning location #################################
    printTime("Select potential spawning locations started")
    predictLocations()
    
    ################################# Prediction spawning location by FBCR #####################################
    printTime("Prediction spawning location by FBCR started")
    FBCR_predict()
    


In [ ]:


for location in locations:
    printTime("Working on "+location)
    source_dir = 'D:\\WHITEBAIT\\'
    source_dir = source_dir+location+'\\SOURCE\\'
    output_dir = 'D:\\WHITEBAIT\\'+location

    if not os.path.exists(output_dir+'\\OUTPUT\\'):
        os.makedirs(output_dir+'\\OUTPUT\\')

    output_dir = output_dir+'\\OUTPUT\\'

    tile_index = source_dir + 'study_area.shp' # source file containing required DEM and DSM file name and a defined study area
    tidelevel_inCM = tide[locations.index(location)]
    
    main()
    printTime("Work on "+location+" is done")
    print('\n')

printTime("Program ended")
print("\n")